# **Project results on the freesurfer surfaces**
This notebook will guide you through the step necessary to project fMRI results on the surfaces that were generated with Freesurfer.

### Initiation - Variables and paths

In [3]:
# what subject?
SUBJ=Danny

# where will the result appear?
OUT=/Users/chris/Desktop/CT2Surf_${SUBJ}
mkdir -p $OUT

# where can we find the epi space the results are in?
# This is usually the reference from the `manual-masks' folder that is used for preprocessing and modelfit in NHP-BIDS.

# Danny
EPI_ROOT=/Users/chris/Documents/MRI_ANALYSIS/NHP-BIDS/manual-masks/sub-danny/func/
EPI=${EPI_ROOT}/sub-danny_ref_func_res-1x1x1.nii.gz
EPI_MASK=${EPI_ROOT}/sub-danny_ref_func_mask_res-1x1x1.nii.gz

# Eddy
#EPI_ROOT=/Users/chris/Documents/MRI_ANALYSIS/NHP-BIDS/manual-masks/final/sub-eddy/ses-20170607b/func/
#EPI=${EPI_ROOT}/ref_func_undist_inData_al_fnirt.nii.gz
#EPI_MASK=${EPI_ROOT}/HiRes_to_T1_mean.nii_shadowreg_Eddy_brainmask.nii.gz

# where is the anatomical you'd like to register to?
# $SUBJECTS_DIR should point to the Freesurfer subjects directory. Adjust your bashrc to export this if it doesn't do so already.
T1=${SUBJECTS_DIR}/${SUBJ}/mri/brainmask.mgz
# and the corresponding white matter segmentation?
WM=${SUBJECTS_DIR}/${SUBJ}/mri/wm.mgz

# where's this notebook on the disk?
NOTEBOOK_PATH=/Users/chris/Documents/MRI_ANALYSIS/NHP-Freesurfer/Notebooks # used to identify example data in a subfolder

In [4]:
# Mask the epi that is used as a reference in functional analysis to extract the brain
fslmaths ${EPI} -mas ${EPI_MASK} ${OUT}/epi_brain.nii.gz
EPI_BRAIN=${OUT}/epi_brain.nii.gz

## Flirt registration of epi to T1
The T1 you pick here is the header adjusted one that formed the based of the surface generation in Freesurfer. We do not have to adjust the header of the epi, because flirt will generally be able to take of the scaling. If this fails for some reason you can still do it manually with `3drefit -xdel 2.0 -ydel 2.0 -zdel 2.0 -keepcen <epi>.nii.gz`. Note that we set voxel sizes to 2 mm in the header in this case. The reason is that for the T1 we had 0.5 mm voxels and adjusted the header to 1 mm, a factor of 2. Our pre-processed epi's have 1 mm voxels, so applying the same factor of 2 means the header info should state that the voxels are 2 mm isotropic.

Be aware that, when you first adjust the header and then do flirt, the final registration matrix will not include this scaling and expect header-adjusted input. Any result you want to warp to the surface will thus first have to be adjusted. You don't have to do this is if the flirt is performed on the non-adjusted epi. 

NB1! One thing to carefully check is whether the x-direction of the voxel order matches between the epi and T1 because if it doesn't we will see left/right flips. With some of our older standard epi's this needs correction (later I corrected it in the reference file for pre-processing). There's a script that does that for you called `swap_xdir_voxels.sh`. You can find it in the `bin` folder of the `Process-NHP-MRI` repository (https://github.com/VisionandCognition/Process-NHP-MRI)

NB2! flirt works significantly better if you include the white matter segmentation (`-wmseg`) of the T1 and phase encoding direction (`-pedir`) of the epi. Check the documentation to find how to code this. If you used Chris Klink's standard epi sequence, your `-pedir` is `-2`.

In [5]:
# get the brain and white matter volumes from freesurfer & convert to nifti
mri_convert ${T1} ${OUT}/brain.nii.gz
mri_convert ${WM} ${OUT}/wm.nii.gz
# calculate the registration
flirt -ref ${OUT}/brain.nii.gz -wmseg ${OUT}/wm.nii.gz -in ${EPI_BRAIN} -out ${OUT}/epi2anat.nii.gz -omat ${OUT}/epi2anat.mat -pedir -2   
# check whether nonlinear registration improves on this result (could be a mess as well)
# fnirt --ref ${OUT}/brain.nii.gz --in ${EPI_BRAIN} --aff ${OUT}/epi2anat.mat --iout ${OUT}/epi2anat_fnirt.nii.gz --inmask ${EPI_MASK} 

mri_convert.bin /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/mri/brainmask.mgz /Users/chris/Desktop/CT2Surf_Danny/brain.nii.gz 
$Id: mri_convert.c,v 1.226 2016/02/26 16:15:24 mreuter Exp $
reading from /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/mri/brainmask.mgz...
TR=0.00, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 0, 0)
j_ras = (0, 0, -1)
k_ras = (0, 1, 0)
writing to /Users/chris/Desktop/CT2Surf_Danny/brain.nii.gz...
mri_convert.bin /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/mri/wm.mgz /Users/chris/Desktop/CT2Surf_Danny/wm.nii.gz 
$Id: mri_convert.c,v 1.226 2016/02/26 16:15:24 mreuter Exp $
reading from /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/mri/wm.mgz...
TR=0.00, TE=0.00, TI=0.00, flip angle=0.00
i_ras = (-1, 0, 0)
j_ras = (0, 0, -1)
k_ras = (0, 1, 0)
writing to /Users/chris/Desktop/CT2Surf_Danny/wm.nii.gz...
ERROR::set_bbr_seg: could not find any boundary points!


## Create the tkregister matrix
Now we will use the flirt registration matrix to create a registration matrix in freesurfer format. To get there, we can use the freesurfer program `tkregister` that allows manual registration between 2 volumes. We will inititate it with the flirt matrix and check whether registration is good. If not, you can make manual adjustments, but flirt probably does a better job than manual attempts. Within `tkregister` you can check the alignment of the two volumes by clicking `compare`. Save the registration matrix as `reg.fsl.dat`.

![tkregister](pics/tkregister.gif "tkregister")

In [6]:
# run the tkregister registration initated with the flirt transform
tkregister2 --mov ${EPI_BRAIN} --targ ${OUT}/brain.nii.gz --fsl ${OUT}/epi2anat.mat --reg ${OUT}/reg.fsl.dat --s ${SUBJ}

tkregister_tcl /media/DATA1/NHP_MRI/freesurfer/tktools/tkregister2.tcl
---- FSL registration matrix --------
 2.02740  -0.01509  -0.00542   41.65104;
 0.00710  -0.00596  -2.05053   185.57065;
 0.02381   2.00621   0.17919   34.92606;
 0.00000   0.00000   0.00000   1.00000;
---------------------------------------
target  volume /Users/chris/Desktop/CT2Surf_Danny/brain.nii.gz
movable volume /Users/chris/Desktop/CT2Surf_Danny/epi_brain.nii.gz
reg file       /Users/chris/Desktop/CT2Surf_Danny/reg.fsl.dat
LoadVol        1
ZeroCRAS       0
$Id: tkregister2.c,v 1.132.2.1 2016/08/02 21:17:29 greve Exp $
Diagnostic Level -1
INFO: loading target /Users/chris/Desktop/CT2Surf_Danny/brain.nii.gz
INFO: changing target type to float
Ttarg: --------------------
-1.00000   0.00000   0.00000   128.00000;
 0.00000   0.00000   1.00000  -128.00000;
 0.00000  -1.00000   0.00000   128.00000;
 0.00000   0.00000   0.00000   1.00000;
INFO: loading movable /Users/chris/Desktop/CT2Surf_Danny/epi_brain.nii.gz
Tmov:

In [7]:
tkReg=${OUT}/reg.fsl.dat 

## Convert volumes to surface representation

In [38]:
# These (re)definitions allow to do this part independent of previous steps
SUBJ=${SUBJ} 
tkReg=${OUT}/reg.fsl.dat 

# Pick your stat-maps
RES_Base=/Users/chris/Documents/MRI_ANALYSIS/NHP-CurveTrace_MRI/ActivationMaps/Danny/res
declare -a Location=(DL DR UL UR)

if false; then
    CT_Base=$RES_Base/ct/vox_uncorr
    LC_Base=$RES_Base/lc/vox_uncorr
    
    LC[1]=$LC_Base/zstat4_reg_1mm.nii.gz
    LC[2]=$LC_Base/zstat5_reg_1mm.nii.gz
    LC[3]=$LC_Base/zstat6_reg_1mm.nii.gz
    LC[4]=$LC_Base/zstat7_reg_1mm.nii.gz

    CT[1]=$CT_Base/zstat12_reg_1mm.nii.gz
    CT[2]=$CT_Base/zstat14_reg_1mm.nii.gz
    CT[3]=$CT_Base/zstat11_reg_1mm.nii.gz
    CT[4]=$CT_Base/zstat13_reg_1mm.nii.gz
fi
    
if true; then
    CT_Base=$RES_Base/ct/clust_fwe
    LC_Base=$RES_Base/lc/clust_fwe

    LC[1]=$LC_Base/clust_zmap4.nii.gz
    LC[2]=$LC_Base/clust_zmap5.nii.gz
    LC[3]=$LC_Base/clust_zmap6.nii.gz
    LC[4]=$LC_Base/clust_zmap7.nii.gz

    CT[1]=$CT_Base/clust_zmap12.nii.gz
    CT[2]=$CT_Base/clust_zmap14.nii.gz
    CT[3]=$CT_Base/clust_zmap11.nii.gz
    CT[4]=$CT_Base/clust_zmap13.nii.gz
fi

HEMI=(lh rh) # allow looping over hemispheres when calculating volume to surface transformations

We can now convert the statistical volumes to surface representations using the `mri_vol2surf` command. Since this essentially brings a 3d result to 2d there is a choice to be made on how/where to sample. The `--projfrac` tells the command where between the WM/GM border (`--projfract 0`) and the pial surface (`projfract 1`) to get the data. These fractions can also be negative (going into the WM) or higher than 1 (beyond the pial surface). Alternatively, you can average along the normal between WM/GM border and pial surface using `--projfrac-avg min max stepsize` or the maximum by using `--projfrac-max min max stepsize`.

In [39]:
# look at the mri_vol2surf documentation for more info
# mri_vol2surf --help

In [40]:
# LOCALIZERS ==
cnt=0
for cc in ${LC[@]}; do
    let cnt++
    # create example surface plot
    for xh in ${HEMI[@]}; do
        mri_vol2surf --trgsubject ${SUBJ} --src $cc --out ${OUT}/${xh}.LC${cnt}_avg.w \
            --out_type paint --projfrac-max 0 1 0.1 --srcreg ${tkReg} --hemi ${xh}
    done
done

INFO: output format is paint
srcvol = /Users/chris/Documents/MRI_ANALYSIS/NHP-CurveTrace_MRI/ActivationMaps/Danny/res/lc/clust_fwe/clust_zmap4.nii.gz
srcreg = /Users/chris/Desktop/CT2Surf_Danny/reg.fsl.dat
srcregold = 0
srcwarp unspecified
surf = white
hemi = lh
trgsubject = Danny
surfreg = sphere.reg
ProjFrac = 0.5
thickness = thickness
reshape = 0
interp = nearest
float2int = round
GetProjMax = 1
INFO: float2int code = 0
Done loading volume
INFO: This REGISTER_DAT transform is valid only for volumes between  COR types with c_(r,a,s) = 0.
Input reg is register.dat
-------- original matrix -----------
-0.49320   0.00371   0.00098  -1.01108;
-0.00172  -0.00144   0.48781   0.09871;
-0.00601  -0.49854   0.04358   1.62712;
 0.00000   0.00000   0.00000   1.00000;
-------- original matrix -----------
Reading surface /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/surf/lh.white
Done reading source surface
Reading thickness /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/surf/lh.thickness
Done
M

In [41]:
# CURVE TRACING ==
cnt=0
for cc in ${CT[@]}; do
    let cnt++
    # create example surface plot
    for xh in ${HEMI[@]}; do
        mri_vol2surf --trgsubject ${SUBJ} --src $cc --out ${OUT}/${xh}.CT${cnt}_avg.w \
            --out_type paint --projfrac-max 0 1 0.1 --srcreg ${tkReg} --hemi ${xh}
    done
done

INFO: output format is paint
srcvol = /Users/chris/Documents/MRI_ANALYSIS/NHP-CurveTrace_MRI/ActivationMaps/Danny/res/ct/clust_fwe/clust_zmap12.nii.gz
srcreg = /Users/chris/Desktop/CT2Surf_Danny/reg.fsl.dat
srcregold = 0
srcwarp unspecified
surf = white
hemi = lh
trgsubject = Danny
surfreg = sphere.reg
ProjFrac = 0.5
thickness = thickness
reshape = 0
interp = nearest
float2int = round
GetProjMax = 1
INFO: float2int code = 0
Done loading volume
INFO: This REGISTER_DAT transform is valid only for volumes between  COR types with c_(r,a,s) = 0.
Input reg is register.dat
-------- original matrix -----------
-0.49320   0.00371   0.00098  -1.01108;
-0.00172  -0.00144   0.48781   0.09871;
-0.00601  -0.49854   0.04358   1.62712;
 0.00000   0.00000   0.00000   1.00000;
-------- original matrix -----------
Reading surface /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/surf/lh.white
Done reading source surface
Reading thickness /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/surf/lh.thickness
Done


In [42]:
# show in tksurfer
xh=rh # switch hemisphere easily
tksurfer ${SUBJ} ${xh} graymid -patch full.patch.flat -overlay ${OUT}/${xh}.CT1_avg.w -overlay-reg ${tkReg}

displaying patch full.patch.flat...
subject is Danny
hemi    is rh
surface is graymid
surfer: current subjects dir: /media/DATA1/NHP_MRI/freesurfer/subjects
surfer: not in "scripts" dir ==> using cwd for session root
surfer: session root data dir ($session) set to:
surfer:     /media/DOCUMENTS/DOCUMENTS/MRI_ANALYSIS/NHP-Freesurfer/Notebooks
checking for nofix files in 'graymid'
Reading image info (/media/DATA1/NHP_MRI/freesurfer/subjects/Danny)
Reading /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/mri/T1.mgz
surfer: Reading header info from /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/mri/T1.mgz
ltMNIreadEx: could not open file /media/DATA1/NHP_MRI/freesurfer/subjects/Danny/mri/transforms/talairach.xfm
No such file or directory
surfer: Talairach xform file not found (ignored)
surfer: vertices=77636, faces=155272
         Talairach coords will be incorrect.
surfer: curvature read: min=-5.511981 max=1.183509
surfer: single buffered window
surfer: tkoInitWindow(Danny)
setting percentil

In [ ]:
# show in tksurfer
xh=lh # switch hemisphere easily
tksurfer ${SUBJ} ${xh} graymid -patch full.patch.flat -overlay ${OUT}/${xh}.CT3_avg.w -overlay-reg ${tkReg}